# Import Dependensi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from numpy import hstack
import tensorflow as tf
from numpy import array
from datetime import datetime, timedelta
import matplotlib as mpl
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Dropout, BatchNormalization, Activation
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

Matplotlib created a temporary cache directory at /var/folders/86/7gxks06s07xch2nsjg8bfzhh0000gn/T/matplotlib-gjz04_nf because the default path (/Users/ingwerludwig/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Import Dataset

In [151]:
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
excel_path = os.path.join(parent_dir, 'Model Welang.xlsx')

In [152]:
df = pd.read_excel(excel_path,'Data')
df.rename(columns={'Unnamed: 0': 'DateTime'}, inplace=True)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.sort_values(by='DateTime',ascending=True,inplace=True)
df

,DateTime,RC,RL,LP,LD
0,2022-09-20 20:00:00,0.00,0.00,0.11,0.32
1,2022-09-20 21:00:00,0.00,0.00,0.11,0.31
2,2022-09-20 22:00:00,0.00,0.00,0.11,0.30
3,2022-09-20 23:00:00,0.00,0.00,0.11,0.29
4,2022-09-21 00:00:00,0.00,0.00,0.11,0.29
...,...,...,...,...,...
8611,2023-09-15 08:00:00,0.00,0.00,0.07,0.21
8612,2023-09-15 09:00:00,0.00,0.00,0.08,0.23
8613,2023-09-15 10:00:00,0.00,0.00,0.09,0.24
8614,2023-09-15 11:00:00,0.00,0.00,0.10,0.24


# Isi Data yang tidak Sekuensial 1 Jam dengan Data Sebelumnya

In [153]:
check_df = df['DateTime']
check_df = pd.DataFrame(check_df)

check_df['TimeDifference'] = check_df['DateTime'].diff()
expected_interval = pd.to_timedelta('1 hour')
check_df['not_sequential'] = check_df['TimeDifference'] != expected_interval

check_df.at[0, 'not_sequential'] = False
filtered_df = check_df[check_df['not_sequential']]
filtered_df

,DateTime,TimeDifference,not_sequential
14,2022-09-21 11:00:00,0 days 02:00:00,True
15,2022-09-21 13:00:00,0 days 02:00:00,True
17,2022-09-21 16:00:00,0 days 02:00:00,True
65,2022-09-23 17:00:00,0 days 02:00:00,True
1386,2022-11-17 21:00:00,0 days 04:00:00,True
2158,2022-12-20 04:00:00,0 days 04:00:00,True
4254,2023-03-17 13:00:00,0 days 02:00:00,True
4303,2023-03-19 17:00:00,0 days 04:00:00,True
4397,2023-03-23 17:00:00,0 days 03:00:00,True
4465,2023-03-26 14:00:00,0 days 02:00:00,True


In [154]:
start_date = df['DateTime'].min()
end_date = df['DateTime'].max()
date_range = pd.date_range(start=start_date, end=end_date, freq='1H')
sequence_df = pd.DataFrame({'DateTime': date_range})

df = pd.merge_asof(sequence_df, df, on='DateTime', direction='backward')

# Forward-fill missing values in the 'Akumulasi Curah Hujan Cendono' column
df['RC'].fillna(method='ffill', inplace=True)
df['RC'] = df['RC'].round(3).astype(float)

df['RL'].fillna(method='ffill', inplace=True)
df['RL'] = df['RL'].round(3).astype(float)

df['LP'].fillna(method='ffill', inplace=True)
df['LP'] = df['LP'].round(3).astype(float)

df['LD'].fillna(method='ffill', inplace=True)
df['LD'] = df['LD'].round(3).astype(float)
df

,DateTime,RC,RL,LP,LD
0,2022-09-20 20:00:00,0.00,0.00,0.11,0.32
1,2022-09-20 21:00:00,0.00,0.00,0.11,0.31
2,2022-09-20 22:00:00,0.00,0.00,0.11,0.30
3,2022-09-20 23:00:00,0.00,0.00,0.11,0.29
4,2022-09-21 00:00:00,0.00,0.00,0.11,0.29
...,...,...,...,...,...
8628,2023-09-15 08:00:00,0.00,0.00,0.07,0.21
8629,2023-09-15 09:00:00,0.00,0.00,0.08,0.23
8630,2023-09-15 10:00:00,0.00,0.00,0.09,0.24
8631,2023-09-15 11:00:00,0.00,0.00,0.10,0.24


In [155]:
check_df = df['DateTime']
check_df = pd.DataFrame(check_df)

check_df['TimeDifference'] = check_df['DateTime'].diff()
expected_interval = pd.to_timedelta('1 hour')
check_df['not_sequential'] = check_df['TimeDifference'] != expected_interval

# Fill the first row with False since it doesn't have a previous row for comparison
check_df.at[0, 'not_sequential'] = False
filtered_df = check_df[check_df['not_sequential']]
filtered_df

,DateTime,TimeDifference,not_sequential


In [7]:
df.to_excel('Fix Data_Model Welang.xlsx')